<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT20/blob/timaslj2/timaslj_lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Matrix factorization**
**Timas Ljungdahl**

In collaboration with **Kristoffer Almroth**

# **Abstract**

In this report, 4 different algorithms were implemented and tested. The algorithms were sparse matrix-vector multiplication for matrices of CRS format, modified Gram-Schmidt iteration for QR-factorization, backwards substitution for solving $Rx = b$ and finally QR eigenvalue algorithm for finding the eigenvalues and eigenvectors for a matrix $A$. All algorithms were implemented and tested with random data and generally generated results with around 10 decimal precision. The results are probably highly affected by floating point error that occur when continuously adding and subtracting numbers of different magnitude. 

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [0]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np
import unittest
import random

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

In this report, systems of linear equations are investigated of the form $Ax = b$
where we want to solve for $x$ given a matrix $A$ and vector $b$. A system of linear equations has an exact solution if A is nonsingular, meaning that A has an inverse, since then $x = A^{-1}b$. In this report we assume that A is a square, nonsingular matrix. 

The direct solution methods implemented in this report are based on factorization of A into easily invertable matrices - diagonal, orthonormal and triangular matrices. The problem of solving for $x$ is summerized below.

$Ax = b$, solve for $x$
1. Factorize $A$ into $QR$, where $Q$ is an orthonormal matrix and $R$ is an upper triangular matrix. This gives us $QRx = b$.
2. Since $Q$ is orthonormal $Q^{-1} = Q^T$. Orthonormal, means that each column vector in the matrix is normalized and orthogonal to each other. Multiplying $Q^{-1}$ on the left on each side, we get $Q^{-1}QRx = Q^{-1}b => Rx = Q^Tb$.
3. We can now easily solve $Rx = Q^Tb$ 

Apart from solving systems of equations, sparse matrix-vector multiplication was implemented for sparse matrices of compressed row storage(CRS)format. Instead of storing all values in the matrix, only the nonzero values are stored in an array ***v***, along with the two index arrays ***col_idx*** and ***row_ptr***. There is an entry in the ***col_idx*** array for each value to indicate the column of the value in the original matrix. The ***row_ptr*** array consists of the index in ***v*** where each row starts. 

The QR eigenvalue algorithm was also implementet which for a real symmetric matrix $A$, returns a unitary matrix $U$ with the eigenvectors of $A$ as column vectors and a upper triangular matrix $T$ with eigenvalues of $A$ in the diagonal. It finds the a Schur factorization such that $A = UTU^{*}$.

# **Methods**

To achieve the goal of solving the system of equations, the first step is to implement an effecient factorization method. In this report, a modified version of Gram-Schmidt iteration is implemented. This method recursively computes an orthonormal matrix $Q$ from $A$ by taking each column vector in $A$ and subtracting its projection onto the already computed orthonormal space in order to compute a new perpendicular vector which is then added to the set of orthonormal vectors. $R$ is then computed as $R = Q^TA$.  

When $Q$ and $R$ have been computed, $b$ is multiplied by $Q^T$ on the left to form a new vector $b_{prim}$ so that $Rx = b_{prim}$. This new system of equations can now be solved with backwards subsitution as $R$ is upper triangular. This is possible since $x_n = b_n/a_{nn}$ which can then be substituted in order to solve for $x_{n-1} = (b_{n-1} - a_{(n-1)(n)}x_n)/a_{(n-1)(n-1)}$. This can be written as:

$x_i = (b_i - \sum_{j=i+1}^{n}a_{ij}x_j)/a_{ii} $

In order to implement these algorithms, pseudocode from the lecture notes provided in class were followed. 

To test the QR factorization method,random $A \epsilon \mathbb{R}^{n \times n}$ were generated with imported numpy methods. The output $Q$ and $R$ of the algorithm were then multiplied together again and asserted to be equal to $A$.

To test the direct solver method, random $A \epsilon \mathbb{R}^{n \times n}$ and $b \epsilon \mathbb{R}^{n}$ were generated and the residuals $|| Ax-b ||$ and $|| x-y ||$ were asserted to be equal to zero. 

To test the QR eigenvalue algorithm, a random real symmetric matrix A was generated and $det(A - \lambda_iI)$ was asserted to be equal to 0. $||Av_i - \lambda_iv_i||$ was also asserted to be equal to 0. The tests follow the definition of eigenvectors and eigenvalues: $Av = \lambda v$ where $v$ is an eigenvector and $\lambda$ is an eigenvalue. 

# **Results**

### **Sparse matrix vector multiplication**

In [255]:
def sparse_matrix_vector_product(x, v, col_idx, row_ptr):
  assert x.size == (len(row_ptr)-1)

  product = np.zeros(len(row_ptr)-1)

  for row in range(len(row_ptr)-1):
    res = 0
    for i in range(row_ptr[row], row_ptr[row+1]):
      res += v[i] * x[col_idx[i]]
    product[row] = res
  
  return product

class Test(unittest.TestCase):
  
  def test_illegal_dimensions(self):
    x = np.array([1,2])
    v = [3,2,2,2,1,1,3,2,1,2,3]
    col_idx = [1,2,4,2,3,3,3,4,5,5,6]
    row_ptr = [1,4,6,7,9,10,12]
    with self.assertRaises(AssertionError):
      sparse_matrix_vector_product(x,v, col_idx,row_ptr)

  def test_against_dense_product(self):
    A = np.array([[3,2,0,2,0,0],
                  [0,2,1,0,0,0],
                  [0,0,1,0,0,0],
                  [0,0,3,2,0,0],
                  [0,0,0,0,1,0],
                  [0,0,0,0,2,3]])
    x = np.array([2,1,2,4,1,3])
    v = [3,2,2,2,1,1,3,2,1,2,3]
    col_idx = [0,1,3,1,2,2,2,3,4,4,5]
    row_ptr = [0,3,5,6,8,9,11]
    np.testing.assert_array_equal(np.matmul(A,x), sparse_matrix_vector_product(x,v,col_idx,row_ptr))

  def test_random_sparse_matrix(self):
    for i in range(100):
      size = random.randint(2, 100) 
      A = np.random.rand(size, size)
      x = np.random.rand(size)
      v = []
      col_idx = []
      row_ptr = []
      # make the random matrix sparse, on average 2/3 of matrix are zeros
      for i in range(size):
        seen = False
        nr_of_zeros = 0
        for j in range(size):
          if not(random.randint(0,3) == 0) and nr_of_zeros < size-1:
            A[i,j] = 0
            nr_of_zeros += 1
          else:
            if not(seen):
              row_ptr.append(len(v))
              seen = True
            v.append(A[i,j])
            col_idx.append(j)
      row_ptr.append(len(v))

      np.testing.assert_array_almost_equal(np.matmul(A,x), sparse_matrix_vector_product(x,v,col_idx,row_ptr), 14)
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.854s

OK


### **QR-factorization**

In [256]:
def qr_factorization(A):
  v = np.copy(A)
  n = A.shape[0]
  R = np.zeros((n,n))
  Q = np.zeros((n,n))
  for i in range(n):
    R[i,i] = np.sqrt(np.dot(v[:,i],v[:,i])) #norm(v)
    Q[:,i] = v[:,i]/R[i,i] #normalize
    for j in range(i+1, n):
      R[i,j] = np.dot(Q[:,i], v[:,j]) #qA
      v[:,j] = np.subtract(v[:,j], R[i,j]*Q[:,i]) #update orthogonal set v
  return Q,R

def frobenius_norm(A):
  res = 0
  for i in A:
    for j in i:
      res += j*j
  return np.sqrt(res)

class Test(unittest.TestCase):

  def test_R_upper_triangular(self):
    size = random.randint(2, 100) 
    A = np.random.rand(size, size)
    _, R = qr_factorization(A)

    for i in range(size):
      for j in range(0, i):
        self.assertEqual(R[i,j], 0)
  
  '''In order for Q to be orthonormal, all column vectors must be normalized
  and orthogonal to one another
  '''
  def test_Q_orthonormal(self):
    for n in range(100):
      size = random.randint(2,100) 
      A = np.random.rand(size, size)
      Q, _ = qr_factorization(A)
      for i in range(size):
        # norm == 1
        self.assertAlmostEqual(np.sqrt(np.dot(Q[:,i],Q[:,i])), 1, 15)
        for j in range(i+1,size):
          #orthogonal
          self.assertAlmostEqual(np.dot(Q[:,i], Q[:,j]), 0, 10)
        
  def test_Q_R_equals_A(self):
    for n in range(100):
      size = random.randint(2,100) 
      A = np.random.rand(size, size)
      Q, R = qr_factorization(A)

      np.testing.assert_array_almost_equal(np.matmul(Q,R), A, 14)
  
  def test_frobenius_norms(self):
    for n in range(100):
      size = random.randint(2,100) 
      A = np.random.rand(size, size)
      Q, R = qr_factorization(A)
      #|| QR-A ||_F
      self.assertAlmostEqual(frobenius_norm(np.matmul(Q,R)) - frobenius_norm(A), 0, 11)

      Q_trans = np.transpose(Q)
      #|| Q^TQ ||
      self.assertAlmostEqual(frobenius_norm(np.matmul(Q_trans, Q)), np.sqrt(size), 14)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 7.398s

OK


### **Backward substitution**

In [257]:
def backward_subsitution(A,b):
  n = b.shape[0]
  x = np.zeros(n)
  x[n-1] = b[n-1]/A[n-1,n-1]
  for i in range(n-2, -1, -1):
    res = 0
    for j in range(i+1, n):
      res += A[i,j]*x[j]
    x[i] = (b[i] - res)/A[i,i]
  return x

def direct_solver(A, b):
  Q, R = qr_factorization(A)
  b_prim = np.zeros(Q.shape[0])

  #Q^t * b
  for j in range(Q.shape[0]):
    b_prim[j] = np.dot(Q[:,j],b)

  return backward_subsitution(R, b_prim)

class Test(unittest.TestCase):

  def test_backward_sub(self):
    for n in range(100):
      size = random.randint(2,100) 
      A = np.random.rand(size, size)
      b = np.random.rand(size)
      #get random upper triangular matrix
      _, R = qr_factorization(A)

      x = backward_subsitution(R, b)
      
      np.testing.assert_array_almost_equal(np.matmul(R,x), b, 11)

  def test_residuals(self):
    for n in range(100):
      size = random.randint(2,100) 
      A = np.random.rand(size, size)
      b = np.random.rand(size)

      y = direct_solver(A, b)

      res_vec = np.matmul(A,y)-b
      
      #||Ax-b||
      self.assertAlmostEqual(np.sqrt(np.dot(res_vec,res_vec)), 0, 10)

      x = np.linalg.solve(A, b)
      diff_vec = x-y

      #||x-y||
      self.assertAlmostEqual(np.sqrt(np.dot(diff_vec,diff_vec)), 0, 7)


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 3.597s

OK


### **Blocked matrix-matrix product**

In [292]:
def qr_algorithm(A):
  n = A.shape[0]
  U = np.identity(n)
  for k in range(10000):
    Q, R = qr_factorization(A)
    A = np.matmul(R, Q)
    U = np.matmul(U, Q)
  return A, U

class Test(unittest.TestCase):

  def test_determinant(self):
    for n in range(100):
      size = random.randint(2,5)
      sym_A = np.zeros((size,size))

      for i in range(size):
        for j in range(size):
          rand_int = random.randint(0,50)
          sym_A[i,j] = rand_int
          sym_A[j,i] = rand_int
      A, U = qr_algorithm(sym_A)
      
      id_matrix = np.identity(size)
      for i in range(size):
        res = np.matmul(sym_A, U[:,i]) - A[i,i]*U[:,i]
        self.assertAlmostEqual(np.sqrt(np.dot(res,res)), 0, 11)
        self.assertAlmostEqual(np.linalg.det(sym_A - (A[i,i]*id_matrix)), 0, 3)

    #for i in range(size):


if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 76.072s

OK


# **Discussion**

All algorithms were implemented and tested with random data several times. The precision of the algorithms differs but generally the output had a precision of around 10 decimals. The precision is probabaly affected by floating point errors that occur when adding and subtracting numbers of different magnitude. The modified Graham-Schmidt iteration, however, mitigates the absorption effect of floating point addition as the orthonormal set is generated recursively and does not rely on the summation of a large set of numbers.    